In [1]:
import numpy as np
from numba import njit

# References

* [Knapsack Problem Dynamic Programming](https://www.youtube.com/watch?v=zRza99HPvkQ)
* [Ekillion Train Example](https://github.com/takemaru/graphillion/wiki/Example-codes#ekillion---weighted-edges-and-vertices)
* [Finding All Knapsack Solutions (Paths) Within a Range of Weight Limits](https://github.com/takemaru/graphillion/issues/66)

In [2]:
@njit
def knapsack(weights, values, capacity, n, lookup):
    """
    Based on these dynamic programming explanations:
    https://www.youtube.com/watch?v=hagBB17_hvg
    https://www.youtube.com/watch?v=zRza99HPvkQ

    Nearly identical code can be found here:
    https://www.geeksforgeeks.org/0-1-knapsack-problem-dp-10/

    Search for "Bottom-up Approach for 0/1 Knapsack Problem"

    Note that the weights and values vectors are assumed to have a zero
    prepended to them. Thus, the index of each item starts at one.
    """
    for i in range(n+1):
        for w in range(capacity+1):
            if i == 0 or w == 0:
                lookup[i, w] = 0
            elif weights[i] <= w:
                lookup[i, w] = max(
                    lookup[i-1, w-weights[i]] + values[i],
                    lookup[i-1, w]
                )
            else:
                lookup[i, w] = lookup[i-1, w]
    return lookup[n, w]

In [3]:
class KNAPSACK:
    def __init__(self, weights, values, capacity):
        self._n = len(weights)  # This is the actual number of items
        self._weights = np.concatenate(([0], weights))
        self._values = np.concatenate(([0], values))
        self._capacity = capacity
        self._lookup = np.empty((self._n+1, self._capacity+1), dtype=np.uint64)
        self._max_value = None
        self._items = []

    def solve(self):
        self._max_value = knapsack(self._weights, self._values, self._capacity, self._n, self._lookup)
        i = self._n
        j = self._capacity
        while i > 0 and j > 0:
            if self._lookup[i, j] == self._lookup[i-1, j]:
                pass  # Item is excluded
            else:
                idx = i - 1
                self._items.append(idx)  # Item is included
                j = j - self._weights[i]
            i = i-1

    @property
    def max_value(self):
        return self._max_value

    @property
    def items(self):
        """
        These are the indices corresponding to the original weights or
        values vectors
        """
        return self._items

In [4]:
weights = [10, 20, 30]
values = [60, 100, 120]
capacity = 50

ks = KNAPSACK(weights, values, capacity)
ks.solve()
assert ks._max_value == 220
assert ks._items == [2, 1]